In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff 
from plotly.grid_objs import Grid, Column 
import colorlover as cl

#####################################################
#####################################################

                ##### STOCK X #####
                     # DATA #
                #### CHALLENGE ####
            
#####################################################
#####################################################

path = 'https://s3.amazonaws.com/stockx-sneaker-analysis/wp-content/uploads/2019/02/StockX-Data-Contest-2019.xlsx'

df = pd.read_excel(path, sheet_name=1)

df.head()

,Order Date,Brand,Sneaker Name,Sale Price,Retail Price,Release Date,Shoe Size,Buyer Region
0,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-Low-V2-Beluga,1097.0,220,2016-09-24,11.0,California
1,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Copper,685.0,220,2016-11-23,11.0,California
2,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Green,690.0,220,2016-11-23,11.0,California
3,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red,1075.0,220,2016-11-23,11.5,Kentucky
4,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red-2017,828.0,220,2017-02-11,11.0,Rhode Island


In [2]:
off_white = df.copy()
off_white = off_white[off_white['Brand'].isin(['Off-White'])]

yeezy = df.copy()
yeezy = yeezy[yeezy['Brand'].isin([' Yeezy'])]

print('Shape of Off-White dataframe : {}'.format(off_white.shape))
print('Shape of Yeezy dataframe : {}'.format(yeezy.shape))

Shape of Off-White dataframe : (27794, 8)
Shape of Yeezy dataframe : (72162, 8)


In [3]:
#gonna do some simple subtraction!
def profit_after_fees(x,y):
    '''
    Taking into account StockX's seller fees at 9.5% maximum and the additional 3% transaction fees.
    Should be 12.5% max 
    https://help.stockx.com/selling-on-stockx/what-are-stockx-selling-fees
    '''
    profit = x - y
    return round((profit * .875),2)

In [4]:
yeezy['Profit After Fees'] = yeezy.apply(lambda x: profit_after_fees(x['Sale Price'],x['Retail Price']), axis=1)
yeezy.head()

,Order Date,Brand,Sneaker Name,Sale Price,Retail Price,Release Date,Shoe Size,Buyer Region,Profit After Fees
0,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-Low-V2-Beluga,1097.0,220,2016-09-24,11.0,California,767.38
1,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Copper,685.0,220,2016-11-23,11.0,California,406.88
2,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Green,690.0,220,2016-11-23,11.0,California,411.25
3,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red,1075.0,220,2016-11-23,11.5,Kentucky,748.12
4,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red-2017,828.0,220,2017-02-11,11.0,Rhode Island,532.00


In [5]:
yeezy['Favorite'] = yeezy.groupby(['Buyer Region','Sneaker Name'])['Sneaker Name'].transform('count')
yeezy.head()

,Order Date,Brand,Sneaker Name,Sale Price,Retail Price,Release Date,Shoe Size,Buyer Region,Profit After Fees,Favorite
0,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-Low-V2-Beluga,1097.0,220,2016-09-24,11.0,California,767.38,115
1,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Copper,685.0,220,2016-11-23,11.0,California,406.88,58
2,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Green,690.0,220,2016-11-23,11.0,California,411.25,58
3,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red,1075.0,220,2016-11-23,11.5,Kentucky,748.12,8
4,2017-09-01,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red-2017,828.0,220,2017-02-11,11.0,Rhode Island,532.00,3


In [6]:
z = yeezy.groupby(['Buyer Region','Sneaker Name'], as_index=False)['Favorite'].max()
z.head()

,Buyer Region,Sneaker Name,Favorite
0,Alabama,Adidas-Yeezy-Boost-350-Low-Pirate-Black-2016,1
1,Alabama,Adidas-Yeezy-Boost-350-Low-V2-Beluga,4
2,Alabama,Adidas-Yeezy-Boost-350-V2-Beluga-2pt0,47
3,Alabama,Adidas-Yeezy-Boost-350-V2-Blue-Tint,47
4,Alabama,Adidas-Yeezy-Boost-350-V2-Core-Black-Red,1


In [7]:
xz = z.pivot('Buyer Region','Sneaker Name','Favorite').fillna(0)
xz.head()

Sneaker Name,Adidas-Yeezy-Boost-350-Low-Moonrock,Adidas-Yeezy-Boost-350-Low-Oxford-Tan,Adidas-Yeezy-Boost-350-Low-Pirate-Black-2015,Adidas-Yeezy-Boost-350-Low-Pirate-Black-2016,Adidas-Yeezy-Boost-350-Low-Turtledove,Adidas-Yeezy-Boost-350-Low-V2-Beluga,Adidas-Yeezy-Boost-350-V2-Beluga-2pt0,Adidas-Yeezy-Boost-350-V2-Blue-Tint,Adidas-Yeezy-Boost-350-V2-Core-Black-Copper,Adidas-Yeezy-Boost-350-V2-Core-Black-Green,Adidas-Yeezy-Boost-350-V2-Core-Black-Red,Adidas-Yeezy-Boost-350-V2-Core-Black-Red-2017,Adidas-Yeezy-Boost-350-V2-Core-Black-White,Adidas-Yeezy-Boost-350-V2-Cream-White,Adidas-Yeezy-Boost-350-V2-Semi-Frozen-Yellow,Adidas-Yeezy-Boost-350-V2-Sesame,Adidas-Yeezy-Boost-350-V2-Zebra,adidas-Yeezy-Boost-350-V2-Butter,adidas-Yeezy-Boost-350-V2-Static,adidas-Yeezy-Boost-350-V2-Static-Reflective
Buyer Region,,,,,,,,,,,,,,,,,,,,
Alabama,0.0,0.0,0.0,1.0,0.0,4.0,47.0,47.0,0.0,0.0,1.0,1.0,5.0,51.0,36.0,24.0,71.0,48.0,23.0,16.0
Alaska,0.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,0.0,0.0,1.0,1.0,0.0,9.0,5.0,3.0,4.0,4.0,5.0,0.0
Arizona,1.0,0.0,1.0,1.0,1.0,5.0,147.0,139.0,1.0,2.0,2.0,10.0,6.0,152.0,62.0,81.0,142.0,164.0,59.0,29.0
Arkansas,1.0,0.0,0.0,0.0,0.0,1.0,20.0,18.0,1.0,0.0,0.0,2.0,1.0,26.0,7.0,10.0,12.0,26.0,13.0,3.0
California,25.0,16.0,28.0,38.0,22.0,115.0,2146.0,1682.0,58.0,58.0,82.0,250.0,142.0,1653.0,587.0,1045.0,1737.0,1810.0,923.0,696.0


In [8]:
xz['Favorite Sneaker'] = xz[['Adidas-Yeezy-Boost-350-Low-Moonrock',
       'Adidas-Yeezy-Boost-350-Low-Oxford-Tan',
       'Adidas-Yeezy-Boost-350-Low-Pirate-Black-2015',
       'Adidas-Yeezy-Boost-350-Low-Pirate-Black-2016',
       'Adidas-Yeezy-Boost-350-Low-Turtledove',
       'Adidas-Yeezy-Boost-350-Low-V2-Beluga',
       'Adidas-Yeezy-Boost-350-V2-Beluga-2pt0',
       'Adidas-Yeezy-Boost-350-V2-Blue-Tint',
       'Adidas-Yeezy-Boost-350-V2-Core-Black-Copper',
       'Adidas-Yeezy-Boost-350-V2-Core-Black-Green',
       'Adidas-Yeezy-Boost-350-V2-Core-Black-Red',
       'Adidas-Yeezy-Boost-350-V2-Core-Black-Red-2017',
       'Adidas-Yeezy-Boost-350-V2-Core-Black-White',
       'Adidas-Yeezy-Boost-350-V2-Cream-White',
       'Adidas-Yeezy-Boost-350-V2-Semi-Frozen-Yellow',
       'Adidas-Yeezy-Boost-350-V2-Sesame', 'Adidas-Yeezy-Boost-350-V2-Zebra',
       'adidas-Yeezy-Boost-350-V2-Butter', 'adidas-Yeezy-Boost-350-V2-Static',
       'adidas-Yeezy-Boost-350-V2-Static-Reflective']].max(axis=1)

In [11]:
l = []
for i in range(0,len(xz)):
    p = (xz == xz['Favorite Sneaker'][i]).idxmax(axis=1)[i]
    l.append(p)

new_df = pd.DataFrame(l,columns={'Sneaker'})
new_df.head()
new_df['State'] = xz.index
new_df.head()

,Sneaker,State
0,Adidas-Yeezy-Boost-350-V2-Zebra,Alabama
1,Adidas-Yeezy-Boost-350-V2-Cream-White,Alaska
2,adidas-Yeezy-Boost-350-V2-Butter,Arizona
3,Adidas-Yeezy-Boost-350-V2-Cream-White,Arkansas
4,Adidas-Yeezy-Boost-350-V2-Beluga-2pt0,California


In [16]:
yeezy_copy = yeezy.copy()
yeezy_copy = yeezy_copy['Buyer Region'].value_counts().reset_index().rename(columns={'index': 'State', 'Buyer Region': 'Total Sales'})

In [18]:
yeezy_df = new_df.copy()
yeezy_df.head()

,Sneaker,State
0,Adidas-Yeezy-Boost-350-V2-Zebra,Alabama
1,Adidas-Yeezy-Boost-350-V2-Cream-White,Alaska
2,adidas-Yeezy-Boost-350-V2-Butter,Arizona
3,Adidas-Yeezy-Boost-350-V2-Cream-White,Arkansas
4,Adidas-Yeezy-Boost-350-V2-Beluga-2pt0,California


In [19]:

ids = [
'AL',
'AK',    
'AZ',
'AR',
'CA',
'CO',
'CT',
'DE',
'DC',
'FL',
'GA',
'HI',
'ID',
'IL',
'IN',
'IA',    
'KS',
'KY',
'LA',
'ME',
'MD',
'MA',
'MI',
'MN',
'MS',
'MO',
'MT',
'NE',
'NV',
'NH',
'NJ',
'NM',
'NY',
'NC',
'ND',
'OH',
'OK',
'OR',
'PA',
'RI',
'SC',
'SD',
'TN',
'TX',
'UT',
'VT',
'VA',
'WA',
'WV',
'WI',
'WY']
ids_l = pd.DataFrame(ids,columns={'IDs'})
ids_l.head()

,IDs
0,AL
1,AK
2,AZ
3,AR
4,CA
